# 查询转换手册

用户查询在作为RAG查询引擎、代理或任何其他流水线的一部分执行之前，可以以多种方式进行转换和分解。

在本指南中，我们将向您展示不同的转换和分解查询的方式，并找到相关工具集。每种技术可能适用于不同的用例！

为了命名的目的，我们将基础流水线定义为“工具”。以下是不同的查询转换：

1. **路由**：保留查询，但识别查询适用的工具的相关子集。将这些工具作为相关选择输出。
2. **查询重写**：保留工具，但以各种不同的方式重写查询，以针对相同的工具执行。
3. **子问题**：将查询分解为针对不同工具（通过它们的元数据标识）的多个子问题。
4. **ReAct代理工具选择**：给定初始查询，识别1）要选择的工具，和2）要在工具上执行的查询。

本指南的目标是向您展示如何将这些查询转换作为**模块化**组件使用。当然，这些组件中的每一个都可以插入到更大的系统中（例如，子问题生成器是我们的`SubQuestionQueryEngine`的一部分）-每个组件的指南都在下面链接中。

看一看，让我们知道您的想法！


In [ ]:
%pip install llama-index-question-gen-openai
%pip install llama-index-llms-openai

In [ ]:
from IPython.display import Markdown, display# 定义显示提示的函数def display_prompt_dict(prompts_dict):    for k, p in prompts_dict.items():        text_md = f"**提示键**：{k}<br>" f"**文本：**<br>"        display(Markdown(text_md))        print(p.get_template())        display(Markdown("<br><br>"))

## 路由

在这个例子中，我们展示了如何使用查询来选择一组相关的工具选项。

我们使用我们的 `selector` 抽象来选择相关的工具 - 它可以是单个工具，也可以是多个工具，这取决于抽象的定义。

我们有四个选择器：(LLM 或函数调用) x (单选或多选) 的组合。


In [ ]:
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

In [ ]:
# pydantic选择器将pydantic对象提供给调用API的函数# 单个选择器（pydantic，函数调用）# selector = PydanticSingleSelector.from_defaults()# 多个选择器（pydantic，函数调用）# selector = PydanticMultiSelector.from_defaults()# LLM选择器使用文本补全端点# 单个选择器（LLM）# selector = LLMSingleSelector.from_defaults()# 多个选择器（LLM）selector = LLMMultiSelector.from_defaults()

In [ ]:
from llama_index.core.tools import ToolMetadata

tool_choices = [
    ToolMetadata(
        name="covid_nyt",
        description=("This tool contains a NYT news article about COVID-19"),
    ),
    ToolMetadata(
        name="covid_wiki",
        description=("This tool contains the Wikipedia page about COVID-19"),
    ),
    ToolMetadata(
        name="covid_tesla",
        description=("This tool contains the Wikipedia page about apples"),
    ),
]

In [ ]:
display_prompt_dict(selector.get_prompts())

**Prompt Key**: prompt<br>**Text:** <br>

Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.
---------------------
{context_list}
---------------------
Using only the choices above and not prior knowledge, return the top choices (no more than {max_outputs}, but only select what is needed) that are most relevant to the question: '{query_str}'


The output should be ONLY JSON formatted as a JSON instance.

Here is an example:
[
    {{
        choice: 1,
        reason: "<insert reason for choice>"
    }},
    ...
]



<br><br>

In [ ]:
selector_result = selector.select(
    tool_choices, query="Tell me more about COVID-19"
)

In [ ]:
selector_result.selections

[SingleSelection(index=0, reason='This tool contains a NYT news article about COVID-19'),
 SingleSelection(index=1, reason='This tool contains the Wikipedia page about COVID-19')]

了解更多关于我们的路由抽象，请访问我们的[专门的路由器页面](https://docs.llamaindex.ai/en/stable/module_guides/querying/router/root.html)。


## 查询重写

在本节中，我们将向您展示如何将查询重写为多个查询。然后，您可以针对检索器执行所有这些查询。

这是高级检索技术中的关键步骤。通过进行查询重写，您可以为[集成检索]和[融合]生成多个查询，从而获得更高质量的检索结果。

与子问题生成器不同，这只是一个提示调用，独立于工具存在。


### 查询重写（自定义）

在这里，我们将向您展示如何使用提示来生成多个查询，使用我们的LLM和提示抽象。


In [ ]:
from llama_index.core import PromptTemplatefrom llama_index.llms.openai import OpenAIquery_gen_str = """\您是一个乐于助人的助手，根据单个输入查询生成多个搜索查询。生成{num_queries}个搜索查询，每行一个，与以下输入查询相关：查询：{query}查询："""query_gen_prompt = PromptTemplate(query_gen_str)llm = OpenAI(model="gpt-3.5-turbo")def generate_queries(query: str, llm, num_queries: int = 4):    response = llm.predict(        query_gen_prompt, num_queries=num_queries, query=query    )    # 假设LLM适当地将每个查询放在一行上    queries = response.split("\n")    queries_str = "\n".join(queries)    print(f"生成的查询：\n{queries_str}")    return queries

In [ ]:
queries = generate_queries("What happened at Interleaf and Viaweb?", llm)

Generated queries:
1. What were the major events or milestones in the history of Interleaf and Viaweb?
2. Who were the founders and key figures involved in the development of Interleaf and Viaweb?
3. What were the products or services offered by Interleaf and Viaweb?
4. Are there any notable success stories or failures associated with Interleaf and Viaweb?


In [ ]:
queries

['1. What were the major events or milestones in the history of Interleaf and Viaweb?',
 '2. Who were the founders and key figures involved in the development of Interleaf and Viaweb?',
 '3. What were the products or services offered by Interleaf and Viaweb?',
 '4. Are there any notable success stories or failures associated with Interleaf and Viaweb?']

For more details about an e2e implementation with a retriever, check out our guides on our fusion retriever:
- [Module Guide](https://docs.llamaindex.ai/en/stable/examples/retrievers/reciprocal_rerank_fusion.html)
- [Build a Fusion Retriever from Scratch](https://docs.llamaindex.ai/en/latest/examples/low_level/fusion_retriever.html)


### 查询重写（使用QueryTransform）

在本节中，我们将向您展示如何使用我们的QueryTransform类进行查询转换。


In [ ]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.llms.openai import OpenAI

In [ ]:
hyde = HyDEQueryTransform(include_original=True)
llm = OpenAI(model="gpt-3.5-turbo")

query_bundle = hyde.run("What is Bel?")

这将生成一个查询包，其中包含原始查询，还包括表示应该被嵌入的查询的`custom_embedding_strs`。


In [ ]:
new_query.custom_embedding_strs

['Bel is a term that has multiple meanings and can be interpreted in various ways depending on the context. In ancient Mesopotamian mythology, Bel was a prominent deity and one of the chief gods of the Babylonian pantheon. He was often associated with the sky, storms, and fertility. Bel was considered to be the father of the gods and held great power and authority over the other deities.\n\nIn addition to its mythological significance, Bel is also a title that was used to address rulers and leaders in ancient Babylon. It was a term of respect and reverence, similar to the modern-day title of "king" or "emperor." The title of Bel was bestowed upon those who held significant political and military power, and it symbolized their authority and dominion over their subjects.\n\nFurthermore, Bel is also a common given name in various cultures around the world. It can be found in different forms and variations, such as Belinda, Isabel, or Bella. As a personal name, Bel often carries connotatio

## 子问题

给定一组工具和用户查询，决定要生成的子问题集合，以及每个子问题应该运行的工具。

我们将通过使用`OpenAIQuestionGenerator`进行示例运行，它依赖于函数调用，还有`LLMQuestionGenerator`，它依赖于提示。


In [ ]:
from llama_index.core.question_gen import LLMQuestionGenerator
from llama_index.question_gen.openai import OpenAIQuestionGenerator
from llama_index.llms.openai import OpenAI

In [ ]:
llm = OpenAI()
question_gen = OpenAIQuestionGenerator.from_defaults(llm=llm)

In [ ]:
display_prompt_dict(question_gen.get_prompts())

**Prompt Key**: question_gen_prompt<br>**Text:** <br>

You are a world class state of the art agent.

You have access to multiple tools, each representing a different data source or API.
Each of the tools has a name and a description, formatted as a JSON dictionary.
The keys of the dictionary are the names of the tools and the values are the descriptions.
Your purpose is to help answer a complex user question by generating a list of sub questions that can be answered by the tools.

These are the guidelines you consider when completing your task:
* Be as specific as possible
* The sub questions should be relevant to the user question
* The sub questions should be answerable by the tools provided
* You can generate multiple sub questions for each tool
* Tools must be specified by their name, not their description
* You don't need to use a tool if you don't think it's relevant

Output the list of sub questions by calling the SubQuestionList function.

## Tools
```json
{tools_str}
```

## User Question
{query_str}



<br><br>

In [ ]:
from llama_index.core.tools import ToolMetadata

tool_choices = [
    ToolMetadata(
        name="uber_2021_10k",
        description=(
            "Provides information about Uber financials for year 2021"
        ),
    ),
    ToolMetadata(
        name="lyft_2021_10k",
        description=(
            "Provides information about Lyft financials for year 2021"
        ),
    ),
]

In [ ]:
from llama_index.core import QueryBundle

query_str = "Compare and contrast Uber and Lyft"
choices = question_gen.generate(tool_choices, QueryBundle(query_str=query_str))

输出是`SubQuestion` Pydantic对象。


In [ ]:
choices

[SubQuestion(sub_question='What are the financials of Uber for the year 2021?', tool_name='uber_2021_10k'),
 SubQuestion(sub_question='What are the financials of Lyft for the year 2021?', tool_name='lyft_2021_10k')]

有关如何以更加打包的方式将其插入到您的RAG流水线中的详细信息，请查看我们的[SubQuestionQueryEngine](https://docs.llamaindex.ai/en/latest/examples/query_engine/sub_question_query_engine.html)。


## 使用ReAct提示进行查询转换

ReAct是一个流行的代理框架，这里我们展示了如何使用核心ReAct提示来转换查询。

我们使用`ReActChatFormatter`来获取用于LLM的输入消息集合。


In [ ]:
from llama_index.core.agent import ReActChatFormatter
from llama_index.core.agent.react.output_parser import ReActOutputParser
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import ChatMessage

In [ ]:
def execute_sql(sql: str) -> str:    """给定一个SQL输入字符串，执行它。"""    # 注意：这是一个模拟函数    return f"执行了 {sql}"def add(a: int, b: int) -> int:    """两个数字相加。"""    return a + btool1 = FunctionTool.from_defaults(fn=execute_sql)tool2 = FunctionTool.from_defaults(fn=add)tools = [tool1, tool2]

在这里，我们获取输入提示消息，以便传递给LLM。看一下！


In [ ]:
chat_formatter = ReActChatFormatter()
output_parser = ReActOutputParser()
input_msgs = chat_formatter.format(
    tools,
    [
        ChatMessage(
            content="Can you find the top three rows from the table named `revenue_years`",
            role="user",
        )
    ],
)
input_msgs

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: execute_sql\nTool Description: execute_sql(sql: str) -> str\nGiven a SQL input string, execute it.\nTool Args: {\'title\': \'execute_sql\', \'type\': \'object\', \'properties\': {\'sql\': {\'title\': \'Sql\', \'type\': \'string\'}}, \'required\': [\'sql\']}\n\n> Tool Name: add\nTool Description: add(a: int, b: int) -> int\nAdd two numbers.\nTool Args: {\'title\': \'add\', \'type\': \'object\', \'properties\': {\'a\': {\'title\': \'A\', \'type\': \'integer\'}, \'b\': {\'title\': \

接下来我们从模型中获取输出。


In [ ]:
llm = OpenAI(model="gpt-4-1106-preview")

In [ ]:
response = llm.chat(input_msgs)

最后，我们使用我们的ReActOutputParser将内容解析为结构化输出，并分析动作输入。


In [ ]:
reasoning_step = output_parser.parse(response.message.content)

In [ ]:
reasoning_step.action_input

{'sql': 'SELECT * FROM revenue_years ORDER BY revenue DESC LIMIT 3'}